In [3]:
from planet4 import markings, dbscan, io, fnotching

# Some regiondata to produce

In [4]:
from planet4 import region_data

In [12]:
savedir='only_on_small_blotch_20px'
dbscanner = dbscan.DBScanner(savedir=savedir, do_large_run=False)
for image_name in ['ESP_012076_0945']:
    print(image_name)
    dbscanner.cluster_image_name(image_name)
    fnotching.fnotch_image_ids(image_name, savedir=savedir)
    fnotching.apply_cut(image_name, savedir=savedir)

ESP_012076_0945


In [30]:
def do_clustering(obsid):
    from planet4 import dbscan, fnotching
    dbscanner = dbscan.DBScanner(split_by_size=False)
    dbscanner.cluster_image_name(obsid)
    fnotching.fnotch_image_ids(obsid)
    fnotching.apply_cut(obsid)
    return obsid

In [ ]:
for obsid in region_data.Inca.season2:
    do_clustering

In [25]:
from nbtools import execute_in_parallel

In [46]:
result = execute_in_parallel(do_clustering, region_data.Ithaca.season3)

In [34]:
from nbtools import display_multi_progress

In [48]:
display_multi_progress(result, region_data.Ithaca.season3)

# Creating obsid files

In [13]:
from planet4.fnotching import get_clusters_in_path

In [14]:
def create_obsid_result_files(obsid, datapath=None):
#     logger.debug("Working on %s", obsid)
    pm = io.PathManager(obsid=obsid, datapath=datapath)
    # the pm.get_obsid_paths method searches for existing image_id folders inside each
    # obsid folder
    folders = list(pm.get_obsid_paths(pm.L1C_folder))
    blotches = []
    fans = []
    for folder in folders:
        pm.id = fnotching.get_id_from_path(folder)
        if pm.final_blotchfile.exists() and pm.final_blotchfile.lstat().st_size > 1:
            blotches.append(pm.final_blotchdf)
        if pm.final_fanfile.exists() and pm.final_fanfile.lstat().st_size > 1:
            fans.append(pm.final_fandf)

    if len(blotches) > 0:
        blotches = pd.concat(blotches, ignore_index=True)
        blotches.dropna(how='all', axis=1, inplace=True)
        blotches.to_csv(pm.obsid_final_blotches_path, index=False)
        print("Created", pm.obsid_final_blotches_path)

    if len(fans) > 0:
        fans = pd.concat(fans, ignore_index=True)
        fans.dropna(how='all', axis=1, inplace=True)
        fans.to_csv(pm.obsid_final_fans_path, index=False)
        print("Created", pm.obsid_final_fans_path)

In [15]:
create_obsid_result_files('ESP_012076_0945', datapath='only_on_small_blotch_20px')

Created /Users/klay6683/Dropbox/data/planet4/only_on_small_blotch_20px/ESP_012076_0945_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/only_on_small_blotch_20px/ESP_012076_0945_fans.csv


In [49]:
        
from pathlib import Path
for obsid in region_data.Ithaca.season2 + region_data.Ithaca.season3:
    create_obsid_result_files(obsid)

Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011350_0945_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011350_0945_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011351_0945_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011351_0945_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011403_0945_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011403_0945_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011404_0945_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011404_0945_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011931_0945_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_011931_0945_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_012063_0945_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/clustering/ESP_012063_0945_fans.csv
Crea

In [17]:
name= "Ithaca"
region = getattr(region_data, name)
for season_no in [2, 3]:
    season = str(season_no)
    bucket = []
    for obsid in getattr(region, f"season{season}"):
        pm = io.PathManager(obsid=obsid)
        df = pd.read_csv(pm.obsid_final_blotches_path)
        df['obsid'] = obsid
        bucket.append(df)

    df = pd.concat(bucket, ignore_index=True)
    df.to_csv(io.analysis_folder() / f"p4_catalog/{name}_season{season}_blotches.csv",
              index=False)